# 문자 단위 RNN (Char RNN) Practice


- reference : https://wikidocs.net/64703

- 입출력의 단위가 단어 레벨이 아니라 문자 레벨로 하여 RNN을 구현한다면, 이를 문자 단위 RNN이라고 한다.
- 문자 단위 RNN을 다대다(many-to-many) 구조로 구현해보자
- "apple"이라는 문자 시퀀스를 입력받으면 "pple!"을 출력하는 RNN 구현

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

문자 시퀀스 KimSeungHee를 입력받으면 handsome! 을 출력하는 RNN을 구현해보자.

# 1. 훈련 데이터 전처리하기

- 입력 데이터와 레이블 데이터에 대해서 문자 집합(vocabulary)를 만든다. 
- 여기서 문자 집합은 중복을 제거한 문자들의 집합이다.

In [78]:
input_str = 'apple'
label_str = 'pple!'

char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)

print("문자 집합 : {}".format(char_vocab))
print("문자 집합의 크기 : {}".format(vocab_size))

문자 집합 : ['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [79]:
# hyper parameter 정의
input_size = vocab_size  # 입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야 한다.
hidden_size = 5
output_size = 5
learning_rate = 0.1
n_epoch = 100

문자 집합에 고유한 정수를 부여한다.

In [80]:
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))  # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


나중에 예측 결과를 다시 문자 시퀀스로 보기 위해서 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만든다.

In [81]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key
    
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑한다.

In [82]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]

print(x_data)  # KimSeungHee에 해당된다
print(y_data)  # handsome! 에 해당된다

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


파이토치의  nn.RNN()은 기본적으로 3차원 텐서를 입력받는다. 배치 차원을 추가해준다.

In [83]:
x_data = [x_data]
y_data = [y_data]

print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔준다

In [84]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


입력 데이터와 레이블 데이터를 텐서로 바꿔준다

In [85]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [86]:
# 텐서 크기 확인
print("train data의 크기 : {}".format(X.shape))
print("label의 크기 : {}".format(Y.shape))

train data의 크기 : torch.Size([1, 5, 5])
label의 크기 : torch.Size([1, 5])


### 2. RNN 모델 구현

In [87]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [88]:
model = RNN(input_size, hidden_size, output_size)

In [89]:
outputs = model(X)
print(outputs.shape)  # [배치 사이즈, 시점(timesteps), 출력의 크기]를 뜻한다.

torch.Size([1, 5, 5])


정확도를 측정할 때에는 이를 모두 펼쳐서 계산하므로, view를 사용하여 배치 차원과 시점 차원을 하나로 만든다

In [90]:
print(outputs.view(-1, input_size).shape)

torch.Size([5, 5])


레이블 데이터의 크기 확인

In [91]:
print(Y.shape)
print(Y.view(-1).shape)  # [1,5]의 크기를 가지는데, 나중에 정확도 측정할 때에는 펼쳐서 계산할 예정

torch.Size([1, 5])
torch.Size([5])


In [92]:
# optimizer, criterion 정의

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

In [93]:
# 100 번의 Epoch 학습

for epoch in range(n_epoch):
    optimizer.zero_grad()
    outputs = model(X)
    
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))  # view를 하는 이유는 Batch 차원 제거를 위해
    
    loss.backward()  
    optimizer.step()
    
    
    # 모델이 실제로 어떻게 예측했는지 확인을 위한 코드
    
    # 최종 예측값인 각 time-step별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result = outputs.data.numpy().argmax(axis=2)
    
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    
    print("epoch:{}/{} loss:{}, prediction:{}, true Y:{}, prediction str:{}".format(
        epoch+1, n_epoch, loss.item(), result, y_data, result_str 
    ))

epoch:1/100 loss:1.6426725387573242, prediction:[[4 4 4 4 4]], true Y:[[4, 4, 3, 2, 0]], prediction str:ppppp
epoch:2/100 loss:1.4400408267974854, prediction:[[4 4 4 4 4]], true Y:[[4, 4, 3, 2, 0]], prediction str:ppppp
epoch:3/100 loss:1.2884645462036133, prediction:[[4 4 4 4 4]], true Y:[[4, 4, 3, 2, 0]], prediction str:ppppp
epoch:4/100 loss:1.117919921875, prediction:[[4 4 4 4 0]], true Y:[[4, 4, 3, 2, 0]], prediction str:pppp!
epoch:5/100 loss:0.9128655195236206, prediction:[[4 4 3 4 0]], true Y:[[4, 4, 3, 2, 0]], prediction str:pplp!
epoch:6/100 loss:0.6886799931526184, prediction:[[4 4 3 4 0]], true Y:[[4, 4, 3, 2, 0]], prediction str:pplp!
epoch:7/100 loss:0.4866153597831726, prediction:[[4 4 3 2 0]], true Y:[[4, 4, 3, 2, 0]], prediction str:pple!
epoch:8/100 loss:0.3441835641860962, prediction:[[4 4 3 2 0]], true Y:[[4, 4, 3, 2, 0]], prediction str:pple!
epoch:9/100 loss:0.2428395003080368, prediction:[[4 4 3 2 0]], true Y:[[4, 4, 3, 2, 0]], prediction str:pple!
epoch:10/100 l